In [12]:
# Description: DQN for FrozenLake-v1 environment
!pip install gymnasium 'stable-baselines3[extra]'

import gymnasium as gym
from stable_baselines3 import DQN

# 创建环境 env
env = gym.make("FrozenLake-v1",
               render_mode=None,
               max_episode_steps=1000,
               is_slippery=True,  # 必须与训练时一致
               map_name="4x4")    # 明确指定地图尺寸

# 创建DQN模型
model = DQN("MlpPolicy", env, verbose=1)  # 使用多层感知器策略, 环境, 显示训练信息
# model = DQN("MlpPolicy", env, verbose=1, device="cpu")

# 训练模型
model.learn(total_timesteps=500000, log_interval=100)  # 训练步数, 每100步记录一次

# 保存模型
model.save("FrozenLake-dqn")

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
    torch (>=1.9.*)
           ~~~~~~^

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.84     |
|    ep_rew_mean      | 0.01     |
|    exploration_rate | 0.987    |
| time/               |          |
|    episodes         | 100      |
|    fps              | 2268     |
|    time_elapsed     | 0        |
|    total_timesteps  | 684      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0127   |
|    n_updates        | 145      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.2      |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.971 

In [16]:
# Description: DQN测试代码 FrozenLake-v1环境（修复版）
# pip install gymnasium 'stable-baselines3[extra]'

import gymnasium as gym
from stable_baselines3 import DQN
import numpy as np

# 必须与训练时完全相同的环境配置
env = gym.make("FrozenLake-v1",
               render_mode="human",
               max_episode_steps=1000,
               is_slippery=True,  # 必须与训练时一致
               map_name="4x4")    # 明确指定地图尺寸

# 加载模型并绑定环境
model = DQN.load("FrozenLake-dqn", env=env)  # 关键修复：显式传递环境参数

# 重置环境获取初始状态
obs, info = env.reset()
print("初始观察值 (位置编码):", obs)
print("环境信息:", info)

total_reward = 0
step_counter = 0
directions = ["Left", "Down", "Right", "Up"]  # FrozenLake的动作映射

while True:
    # 将numpy数组转换为整数索引（关键修复）
    current_state = int(np.argmax(obs) if isinstance(obs, np.ndarray) else obs)
    
    # 预测动作 (确定性策略)
    action, _states = model.predict(obs, deterministic=True)
    
    # 显示决策信息
    print(f"\nStep {step_counter}:")
    print("当前状态编码:", current_state)
    print("预测动作:", directions[action], f"(动作编号: {action})")
    
    # 执行动作
    obs, reward, terminated, truncated, info = env.step(action)
    
    # 更新统计
    total_reward += reward
    step_counter += 1
    
    # 检查终止条件
    if terminated or truncated:
        print("\n==== Episode 结束 ====")
        print("总步数:", step_counter)
        print("累计奖励:", total_reward)
        print("终止原因:", "到达目标" if terminated else "步数限制")
        break

env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
初始观察值 (位置编码): 0
环境信息: {'prob': 1}

Step 0:
当前状态编码: 0
预测动作: Left (动作编号: 0)


TypeError: unhashable type: 'numpy.ndarray'

In [7]:
import gymnasium as gym
from stable_baselines3 import DQN

# 创建环境
env = gym.make("FrozenLake-v1", render_mode="human")  # 显示图像

# 加载模型
model = DQN.load("FrozenLake-dqn")

# 重置环境, 获得第一个观察值和信息
obs, info = env.reset()  # 重置环境并返回初始观测值
print("重置环境后的观察：", obs)
print("重置环境后的信息：", info)

total_reward = 0  # 总奖励
while True:
    # 预测动作
    action, _states = model.predict(obs, deterministic=True)  # 确定性策略，不使用随机性
    # 打印动作
    print(["Left", "Right"][int(action)])

    # 执行动作
    obs, reward, terminated, truncated, info = env.step(action)

    # 计算总奖励
    total_reward += reward

    # 如果终止或截断: 显示总奖励, 信息, 截断, 终止
    if terminated or truncated:
        # obs, info = env.reset()
        print(f"reward: {total_reward}")
        print(f"truncated: {truncated}")
        print(f"terminated: {terminated}")
        break  # exit(0)

重置环境后的观察： 0
重置环境后的信息： {'prob': 1}
Left


TypeError: unhashable type: 'numpy.ndarray'